In [13]:
from mgca.datasets.classification_dataset import MIMICImageDataset

import torch
import torch.nn as nn

import numpy as np

In [2]:
image_dataset = MIMICImageDataset(split="train", transform=None, data_pct=1.0, img_type="Frontal", imsize=256)

In [14]:
for col in image_dataset.df.columns[2:]:
    print(np.unique(image_dataset.df[col]))

['train']
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[-1.  0.  1.]
[-1.  0.  1.]
[-1.  0.  1.]
[-1.  0.  1.]
[0. 1.]
[0. 1.]
[-1.  0.  1.]
[-1.  0.  1.]
[-1.  0.  1.]
[-1.  0.  1.]


In [ ]:
torch.manual_seed(42)

num_classes = 3

model = nn.Sequential(
    nn.Linear(128+10, )
)